#K-means Clustering in Python

`KMeans(n_clusters=2)` initializes a k-means model with 2 clusters. Parameters are in scikit-learn docs. Fit the model using `model.fit(df)`, where df is a dataframe.  

Commonly used attributes are:

* **cluster_centers_** -- finds the centroid of each cluster
* **labels_** -- finds the labels of all instances
* **inertia_** -- finds the within-cluster sum of squares for the entire dataset

The Python code below clusters eruptions at Old Faithful based on the waiting time between eruptions and the duration of each eruption.

In [ ]:
# Import packages and functions
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.cluster import KMeans

In [ ]:
# Load dataset
geyser = pd.read_csv('https://raw.githubusercontent.com/mh2t/CS6140/main/data/oldfaithful.csv')
geyser

In [ ]:
# Visual exploration
p = sns.scatterplot(data=geyser, x='Eruption', y='Waiting')
p.set_xlabel('Eruption time (min)', fontsize=14)
p.set_ylabel('Waiting time (min)', fontsize=14)

In [ ]:
# Initialize a k-means model with k=2
kmModel = KMeans(n_clusters=2)

# Fit the model
kmModel = kmModel.fit(geyser)

# Save the cluster centroids
centroids = kmModel.cluster_centers_
centroids[1]

In [ ]:
# Save the cluster assignments
clusters = kmModel.fit_predict(geyser[['Eruption', 'Waiting']])

# View the clusters for the first five instances
clusters[0:5]

In [ ]:
# Plot clusters
p = sns.scatterplot(
    data=geyser, x='Eruption', y='Waiting', hue=clusters, style=clusters
)
p.set_xlabel('Eruption time (min)', fontsize=14)
p.set_ylabel('Waiting time (min)', fontsize=14)

# Add centroid for cluster 0
plt.scatter(x=centroids[0, 0], y=centroids[0, 1], c='black')

# Add centroid for cluster 1
plt.scatter(x=centroids[1, 0], y=centroids[1, 1], c='black', marker='X')

In [ ]:
# Fit k-means clustering with k=1,...,5 and save WCSS for each
WCSS = []
k = [1, 2, 3, 4, 5]
for j in k:
    kmModel = KMeans(n_clusters=j)
    kmModel = kmModel.fit(geyser)
    WCSS.append(kmModel.inertia_)

In [ ]:
# Plot the WCSS for each cluster
ax = plt.figure().gca()
plt.plot(k, WCSS, '*-')
plt.xlabel('Number of clusters (k)', fontsize=14)
plt.ylabel('Within-cluster sum of squares (WCSS)', fontsize=14)

#Agglomerative Clustering in Python

`linkage()` in agglomerative clustering connects data. Key parameters are method (e.g., single, complete) for similarity and metric (e.g., Euclidean) for instance distance. More details are in scipy's hierarchical clustering [docs](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html). `dendrogram()` makes dendrograms from dataframes; [scipy's guide](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) covers its parameters.  

For clustering, distance matrices can be useful. Agglomerative clustering accepts them using `squareform` from `spatial.distance` to input a distance matrix.  

The Python code below uses agglomerative clustering to cluster species based on differences in the cytochrome c protein.

In [ ]:
# Import packages and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

# Load the dataset
cytochrome = pd.read_csv('https://raw.githubusercontent.com/mh2t/CS6140/main/data/cytochrome.csv', header=None, usecols=range(1, 14))
cytochrome

In [ ]:
# Add labels for each species and save as a data frame
species = [
    "Human",
    "Monkey",
    "Horse",
    "Cow",
    "Dog",
    "Whale",
    "Rabbit",
    "Kangaroo",
    "Chicken",
    "Penguin",
    "Duck",
    "Turtle",
    "Frog",
]

pd.DataFrame(data=cytochrome.to_numpy(), index=species, columns=species)

In [ ]:
# Format the data as a distance matrix
# In this case, the data already represents distance between points (species)
differences = squareform(cytochrome)

In [ ]:
# Define a clustering model with single linkage
clusterModel = linkage(differences, method='single')

# Create the dendrogram
dendrogram(clusterModel, labels=species, leaf_rotation=90)

# Plot the dendrogram
plt.ylabel('Amino acid differences', fontsize=14)
plt.yticks(np.arange(0, 11, step=1))
plt.xlabel('Species', fontsize=14)
plt.title('Single linkage clustering', fontsize=16)
plt.show()

#DBSCAN in Python

DBSCAN function runs DBSCAN algorithm on vectors/distances. It needs eps and min_samples; defaults are 0.5 and 5. More parameters/values are in [scikit-learn docs](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).  

For varied feature units, standardize using `StandardScaler` from `sklearn.preprocessing`.  

The Python code below uses DBSCAN clustering to model homes based on sales price and square footage.

In [ ]:
# Import packages and functions
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import DBSCAN
from numpy import where
from sklearn.preprocessing import StandardScaler

# Load the dataset
homes = pd.read_csv('https://raw.githubusercontent.com/mh2t/CS6140/main/data/homes.csv')

homes

In [ ]:
# Create a smaller data frame with two variables: Price and Floor
homes_pf = homes[['Price', 'Floor']]
homes_pf.describe()

In [ ]:
# Define a scaler to transform values
scaler = StandardScaler()

# Apply scaler and view result
homes_scaled = pd.DataFrame(scaler.fit_transform(homes_pf), columns=['Price', 'Floor'])
homes_scaled.describe()

In [ ]:
# Initialize DBSCAN model
# Setting a large epsilon will cluster all "middle" values and detect outliers
dbscanModel = DBSCAN(eps=1, min_samples=12)

# Fit the model
dbscanModel = dbscanModel.fit(homes_scaled)

In [ ]:
# Predict clusters
clusters = dbscanModel.fit_predict(homes_scaled)
clusters = pd.Categorical(clusters)
clusters

In [ ]:
# Visualize scaled outliers
p = sns.scatterplot(data=homes_scaled, x='Floor', y='Price', hue=clusters)
p.set_xlabel('Scaled floor', fontsize=14)
p.set_ylabel('Scaled price', fontsize=14)

In [ ]:
# Points where the prediction is -1 are considered outliers
outliers_scaled = homes_scaled[clusters == -1]
outliers_scaled

In [ ]:
# Outliers on original scale (price and square footage in thousands)
outliers_unscaled = homes[clusters == -1]
outliers_unscaled

In [ ]:
# Visualize outliers on original scale
p = sns.scatterplot(data=homes, x='Floor', y='Price', hue=clusters)
p.set_xlabel('Floors', fontsize=14)
p.set_ylabel('Price', fontsize=14)

#Factor Analysis in Python  

Scikit-learn's `decomposition` offers PCA and FactorAnalysis. Both need n_components; default is input features count. More details in [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) and [FactorAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html#sklearn.decomposition.FactorAnalysis) docs.  

The factor loading matrix can be obtained using the code `pca.components_.T * np.sqrt(pca.explained_variance_)`.  

The Python code below applies factor analysis to the rock dataset.

In [ ]:
# Load the pandas package
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# Load the rock.csv dataset
rock = pd.read_csv('https://raw.githubusercontent.com/mh2t/CS6140/main/data/rock.csv')

In [ ]:
# Display the rock dataframe
rock

In [ ]:
# Display the correlation matrix using a heatmap
plt.figure(figsize=(4, 4))
sns.heatmap(rock.corr(), cmap="YlGnBu", annot=True)

In [ ]:
# Create a scatter plot using perimeter and area
plt.figure(figsize=(4, 4))
plt.scatter(rock['Perimeter'], rock['Area'])
plt.xlabel('Perimeter', fontsize=14)
plt.ylabel('Area', fontsize=14)

In [ ]:
# Create a scatter plot with a linear regression line
model = st.linregress(rock['Perimeter'], rock['Area'])
plt.figure(figsize=(4, 4))
plt.scatter(rock['Perimeter'], rock['Area'])
x = np.linspace(0, 5000, 10000)
y = model[0] * x + model[1]
plt.plot(x, y, '-r', linewidth=2.5)
plt.xlabel('Perimeter', fontsize=14)
plt.ylabel('Area', fontsize=14)

In [ ]:
# Scale the data
scaler = StandardScaler()
rock = pd.DataFrame(
    scaler.fit_transform(rock), columns=['Area', 'Perimeter', 'Shape', 'Permeability']
)

In [ ]:
# Initialize and fit a PCA model on the rock data
pcaModel = PCA(n_components=4)
pcaModel.fit(rock)

In [ ]:
# Display the components
pcaModel.components_

In [ ]:
# Display the explained variance (eigenvalues)
pcaModel.explained_variance_

In [ ]:
# Show the factor loadings
pcaModel.components_.T * np.sqrt(pcaModel.explained_variance_)

In [ ]:
# Create a scree plot
xint = range(0, 5)
plt.xticks(xint)
plt.plot([1, 2, 3, 4], pcaModel.explained_variance_, 'b*-')
plt.xlabel('Factors', fontsize='14')
plt.ylabel('Eigenvalues', fontsize='14')